In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
"""
dash==2.0.0
dash-core-components==2.0.0
dash-html-components==2.0.0
dash-table==5.0.0
pandas==1.2.4
plotly==5.4.0
gunicorn==20.1.0
"""

'\ndash==2.0.0\ndash-core-components==2.0.0\ndash-html-components==2.0.0\ndash-table==5.0.0\npandas==1.2.4\nplotly==5.4.0\ngunicorn==20.1.0\n'

In [11]:
#For the bubble map
from datetime import date
from datetime import timedelta
from datetime import datetime,timezone
#today = datetime.today().strftime('%d-%m-%Y')
today = date.today()
yesterday = today - timedelta(days = 1)
yesterday = yesterday.strftime('%m-%d-%Y') 
two_days_ago = today - timedelta(days=2)
two_days_ago = two_days_ago.strftime('%Y-%m-%d')
two_days_ago_url = today - timedelta(days=2)
two_days_ago_url = two_days_ago_url.strftime('%m-%d-%Y')
now_utc = datetime.now(timezone.utc).strftime('%H') 
now_here = datetime.now().strftime('%H') 

if now_utc > '06' : 
    daily_url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{yesterday}.csv?raw=true'
else: 
    daily_url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{two_days_ago_url}.csv?raw=true'
    
    

daily_df = pd.read_csv(url)
daily_df.loc[daily_df["Country_Region"] == "Germany"]
now_utc > '00'


True

In [104]:
pop_url = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv?raw=true'
pop_df= pd.read_csv(pop_url)
pop_df= pop_df[(pop_df['Province_State'].isnull())]
pop_df = pop_df[["Country_Region","Population","iso3"]]
pop_df

continent_url = 'https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv?raw=true'
continent_df = pd.read_csv(continent_url)
continent_df = continent_df[["alpha-3","region","sub-region"]]
continent_df = continent_df.rename(columns={'alpha-3': 'iso3'})
#pop_df = pd.merge(pop_df,continent_df, on="iso3")
pop_df
continent_df
#pop_df.compare(continent_df)

,iso3,region,sub-region
0,AFG,Asia,Southern Asia
1,ALA,Europe,Northern Europe
2,ALB,Europe,Southern Europe
3,DZA,Africa,Northern Africa
4,ASM,Oceania,Polynesia
...,...,...,...
244,WLF,Oceania,Polynesia
245,ESH,Africa,Northern Africa
246,YEM,Asia,Western Asia
247,ZMB,Africa,Sub-Saharan Africa


In [15]:
# calculate incident rate 
#danach vaccine per country 
daily_url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{yesterday}.csv?raw=true'
daily_df = pd.read_csv(daily_url)
countries_df = daily_df[["Country_Region", "Confirmed", "Deaths"]]

countries_df = (countries_df.groupby("Country_Region").sum(
).sort_values(by="Confirmed", ascending=False).reset_index())


pop_url = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv?raw=true'
pop_df= pd.read_csv(pop_url)
pop_df= pop_df[(pop_df['Province_State'].isnull())]
pop_df = pop_df[["Country_Region","Population","iso3"]]
pop_df

continent_url = 'https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv?raw=true'
continent_df = pd.read_csv(continent_url)
continent_df = continent_df[["alpha-3","region","sub-region"]]
continent_df = continent_df.rename(columns={'alpha-3': 'iso3'})
pop_df = pd.merge(pop_df,continent_df, on="iso3")
countries_df = pd.merge(countries_df, pop_df, on="Country_Region")

daily_time_url ='https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
daily_time_df = pd.read_csv(daily_time_url)
daily_time_df = daily_time_df.drop(["Lat","Long"], axis =1)

daily_time_df_AuCaCh = daily_time_df.dropna(subset=['Province/State'])
daily_time_df_AuCaCh = daily_time_df_AuCaCh.groupby("Country/Region").sum()
daily_time_df_AuCaCh = daily_time_df_AuCaCh.loc[['Australia','Canada','China'], :]
daily_time_df_AuCaCh = daily_time_df_AuCaCh.reset_index()
daily_time_df= daily_time_df[(daily_time_df['Province/State'].isnull())]
daily_time_df = daily_time_df.drop(["Province/State"], axis =1)

daily_time_df = daily_time_df.append(daily_time_df_AuCaCh, ignore_index=True)
daily_time_df = daily_time_df.iloc[:, list(range(1)) + list(range(-8, 0))]
daily_time_df["Cases_in_last_7_days"] = (daily_time_df.iloc[:, -1] - daily_time_df.iloc[:, -8])
daily_time_df["Yesterday_Confirmed"] =  (daily_time_df.iloc[:, -2] - daily_time_df.iloc[:, -3])
daily_time_df = daily_time_df.iloc[:, list(range(1)) + list(range(-2, 0))]
daily_time_df = daily_time_df.rename(columns={'Country/Region': 'Country_Region'})
countries_df = pd.merge(countries_df, daily_time_df,on="Country_Region", how='inner')
countries_df["7_Days_Incidence_Rate"] = (countries_df["Cases_in_last_7_days"]/ (countries_df["Population"]/100000))

vac_url = 'https://github.com/govex/COVID-19/blob/master/data_tables/vaccine_data/global_data/vaccine_data_global.csv?raw=true'
df_vaccine = pd.read_csv(vac_url)
df_vaccine = df_vaccine[(df_vaccine['Province_State'].isnull())]
df_vaccine = df_vaccine[["Country_Region","People_fully_vaccinated"]]

countries_df = pd.merge(countries_df, df_vaccine,on="Country_Region", how='inner')
countries_df = countries_df.sort_values(by="7_Days_Incidence_Rate", ascending=False)
countries_df['Population'] =countries_df['Population'].astype(int)
countries_df['7_Days_Incidence_Rate'] =countries_df['7_Days_Incidence_Rate'].astype(int)
countries_df



vac_owid_url ='https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations.csv?raw=true'
df_vac_owid = pd.read_csv(vac_owid_url)
df_vac_owid = df_vac_owid[["location","date","people_fully_vaccinated"]]
df_vac_owid = df_vac_owid.loc[df_vac_owid["date"] == two_days_ago]
df_vac_owid_south_korea = df_vac_owid.loc[df_vac_owid["location"] == "South Korea"]
df_vac_owid_south_korea = df_vac_owid_south_korea[["people_fully_vaccinated"]]



loc_korea_vac_cell = countries_df.loc[countries_df.index[countries_df["Country_Region"] == "Korea, South"], 'People_fully_vaccinated'].item()
if pd.isnull(loc_korea_vac_cell) is True:
    countries_df[countries_df["Country_Region"] == "Korea, South"]= countries_df[countries_df["Country_Region"] == "Korea, South"].replace(to_replace =np.nan,
                 value =df_vac_owid_south_korea.iat[0,0])
else:
    pass

countries_df[countries_df["Country_Region"] == "Korea, South"]
#count = countries_df['People_fully_vaccinated'].isna().sum()
#print(count)
countries_df
#countries_df[countries_df['Country_Region'=='China']]
countries_df[countries_df.isna().any(axis=1)]
countries_df = countries_df.dropna()
#countries_df.index[countries_df["Country_Region"] == "Korea, South"]
countries_df['People_fully_vaccinated'] =countries_df['People_fully_vaccinated'].astype('Int64')


countries_df["Fully_vaccinated_percent"] = (countries_df["People_fully_vaccinated"]/ (countries_df["Population"]))*100
countries_df['Fully_vaccinated_percent'] = countries_df['Fully_vaccinated_percent'].astype('float64')
countries_df['Fully_vaccinated_percent']  = countries_df['Fully_vaccinated_percent'].apply(lambda x: round(x,2))
countries_df.loc[countries_df["Country_Region"] == "China"]
countries_df['sub-region'].unique()
countries_df

,Country_Region,Confirmed,Deaths,Population,iso3,region,sub-region,Cases_in_last_7_days,Yesterday_Confirmed,7_Days_Incidence_Rate,People_fully_vaccinated,Fully_vaccinated_percent
148,Andorra,20136,134,77265,AND,Europe,Southern Europe,1321,696,1709,50333,65.14
167,San Marino,6719,94,33938,SMR,Europe,Southern Europe,411,255,1211,23750,69.98
175,Liechtenstein,5512,68,38137,LIE,Europe,Western Europe,459,111,1203,25418,66.65
33,Slovakia,1295909,15516,5459643,SVK,Europe,Eastern Europe,51547,6657,944,2376047,43.52
57,Denmark,576855,3043,5837213,DNK,Europe,Northern Europe,47921,8314,820,4496437,77.03
...,...,...,...,...,...,...,...,...,...,...,...,...
186,Samoa,3,0,196130,WSM,Oceania,Polynesia,0,0,0,121539,61.97
179,Sao Tome and Principe,3733,57,219161,STP,Africa,Sub-Saharan Africa,0,0,0,48928,22.33
182,Solomon Islands,20,0,652858,SLB,Oceania,Melanesia,0,0,0,52319,8.01
153,Tajikistan,17493,125,9537642,TJK,Asia,Central Asia,0,0,0,2592830,27.19


In [117]:
top10_table = countries_df.head(10)
top10_table = top10_table[["Country_Region","7_Days_Incidence_Rate"]]
top10_table = top10_table.rename(columns={'Country_Region': 'Country'})
top10_table 

,Country,7_Days_Incidence_Rate
175,Liechtenstein,1206
150,Andorra,1047
34,Slovakia,994
21,Czechia,852
58,Denmark,786
25,Belgium,785
40,Switzerland,772
167,San Marino,727
18,Netherlands,707
54,Ireland,630


In [105]:
choro_vaccine_map = px.choropleth(countries_df, locations="Country_Region",
                    locationmode="country names",
                    color="Fully_vaccinated_percent",
                    template="plotly_dark",
                    projection="natural earth",
                    hover_name="Country_Region",
                    hover_data={
                                "Population": ":,",
                                "People_fully_vaccinated": ":,",
                                 "Country_Region": False
                            },
                    color_continuous_scale=px.colors.sequential.Oryel)

choro_vaccine_map.update_layout(
    margin=dict(l=0, r=0, t=50, b=50),
    title={
        'text': "% of the population fully vaccinated against COVID-19",
        'y':0.97,
        'x':0.424,
        'xanchor': 'center',
        'yanchor': 'top'},
    title_font=dict(size=28))
choro_vaccine_map.show()

In [106]:
import plotly.express as px

bubble_map = px.scatter_geo(countries_df,
                            size="7_Days_Incidence_Rate",
                            hover_name="Country_Region",
                            color="7_Days_Incidence_Rate",
                            locations="Country_Region",
                            locationmode="country names",
                            size_max=30,
                            template="plotly_dark",
                            color_continuous_scale=px.colors.sequential.dense,
                            projection="natural earth",
                            hover_data={
                                "Confirmed": ":,",
                                "Deaths": ":,",
                                "Population": ":,",
                                "Yesterday_Confirmed": ":,",
                                 "Country_Region": False
                            })
bubble_map.update_layout(
    margin=dict(l=0, r=0, t=50, b=50),
    title={
        'text': "7 Days COVID-19 Incidence Rate By Country",
        'y':0.97,
        'x':0.424,
        'xanchor': 'center',
        'yanchor': 'top'},
    title_font=dict(size=28))
bubble_map

In [109]:
#totals_df = daily_df[["Confirmed","Deaths"]].sum().reset_index(name="count")

totals_df = countries_df[["Confirmed","Deaths","People_fully_vaccinated"]].sum().reset_index(name="count")
totals_df = totals_df.rename(columns={'index':"condition"})

df_pop_sum = pd.DataFrame(
    {
        "condition": ["Population"],
        "count": [7794592576.0]
    }
)

totals_df = totals_df.append(df_pop_sum, sort=False) #7794592576.0
totals_df = totals_df.reset_index()
totals_df = totals_df.drop('index', 1)
totals_df['count'] =totals_df['count'].astype('Int64')
totals_df 
totals_df_bar = totals_df.iloc[2:4]


bars_graph = px.bar(totals_df_bar,
                    x="condition",
                    y="count",
                    template="plotly_dark",
                    hover_data={'count': ":,"},
                    labels={"condition": "Condition", "count": "Count", "color": "Condition"})
bars_graph.update_traces(marker_color=["#ff5e57", "#0be881"])
bars_graph.update_layout(
    margin=dict(l=0, r=0, t=50, b=50),
    title={
        'text': "Number of People Fully Vaccinated and World Population",
        'y':0.97,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    title_font=dict(size=28))
bars_graph

,condition,count
0,Confirmed,270021051
1,Deaths,5302690
2,People_fully_vaccinated,3633487663
3,Population,7794592576


,condition,count
2,People_fully_vaccinated,3633487663
3,Population,7794592576


In [ ]:
vac_ti_url = 'https://github.com/govex/COVID-19/blob/master/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_global.csv?raw=true'
df_vaccine_time = pd.read_csv(vac_ti_url)
df_vaccine_time = df_vaccine_time[["Country_Region","Date","People_fully_vaccinated"]]
df_vaccine_time 

In [ ]:
vac_url = 'https://github.com/govex/COVID-19/blob/master/data_tables/vaccine_data/global_data/vaccine_data_global.csv?raw=true'
df_vaccine = pd.read_csv(vac_url)
df_vaccine = df_vaccine[(df_vaccine['Province_State'].isnull())]
df_vaccine = df_vaccine[["Country_Region","Date","People_fully_vaccinated"]]
df_vaccine_world = df_vaccine[df_vaccine["Country_Region"] == "World"]
df_vaccine_world = df_vaccine_world[["People_fully_vaccinated"]]
df_vaccine_world = df_vaccine_world.rename(columns={'People_fully_vaccinated': 'count'})
df_vaccine_world.insert(0, "condition", ["Fully vaccinated"], True)
df_vaccine_world

In [ ]:
daily_url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{yesterday}.csv?raw=true'
daily_df = pd.read_csv(daily_url)
daily_df

In [121]:
url_time_confirmed ='https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
time_confirmed_df = pd.read_csv(url_time_confirmed)
time_confirmed_df 
time_confirmed_df = time_confirmed_df.drop(["Province/State","Country/Region","Lat","Long"], axis =1).sum().reset_index(name="total")
time_confirmed_df


,index,total
0,1/22/20,557
1,1/23/20,655
2,1/24/20,941
3,1/25/20,1434
4,1/26/20,2118
...,...,...
687,12/9/21,268558598
688,12/10/21,269250556
689,12/11/21,269739121
690,12/12/21,270159882


In [25]:
daily_time_url ='https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
daily_time_df = pd.read_csv(daily_time_url)
daily_time_df = daily_time_df.drop(["Lat","Long"], axis =1)
daily_time_df= daily_time_df[(daily_time_df['Province/State'].isnull())]
daily_time_df = daily_time_df.drop(["Province/State"], axis =1)
daily_time_df

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,12/4/21,12/5/21,12/6/21,12/7/21,12/8/21,12/9/21,12/10/21,12/11/21,12/12/21,12/13/21
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,157431,157445,157499,157508,157542,157585,157858,157858,157858,157648
1,Albania,0,0,0,0,0,0,0,0,0,...,201402,201730,201902,202295,202641,202863,203215,203524,203787,203925
2,Algeria,0,0,0,0,0,0,0,0,0,...,211297,211469,211662,211859,212047,212224,212434,212652,212848,213058
3,Andorra,0,0,0,0,0,0,0,0,0,...,18010,18010,18631,18815,18815,19272,19440,19440,19440,19440
4,Angola,0,0,0,0,0,0,0,0,0,...,65244,65259,65259,65301,65332,65346,65371,65397,65404,65404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,Vietnam,0,2,2,2,2,2,2,2,2,...,1294778,1309092,1323683,1337523,1352122,1367433,1382272,1398413,1413051,1428428
276,West Bank and Gaza,0,0,0,0,0,0,0,0,0,...,461467,461467,462219,462621,462958,463277,463573,463573,463573,464341
277,Yemen,0,0,0,0,0,0,0,0,0,...,10021,10025,10034,10043,10047,10047,10056,10063,10069,10072
278,Zambia,0,0,0,0,0,0,0,0,0,...,210294,210312,210327,210374,210436,210562,210724,210959,211077,211234


In [33]:
def make_country_confirmed_df(country):
    url_time_confirmed = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
    df = pd.read_csv(url_time_confirmed)
    df = df.loc[df["Country/Region"] == country]
    df = df.drop(columns=["Province/State", "Country/Region",
                 "Lat", "Long"]).sum().reset_index(name="Confirmed")
    df = df.rename(columns={"index": "date"})
    return df
make_country_confirmed_df("Germany")

,date,Confirmed
0,1/22/20,0
1,1/23/20,0
2,1/24/20,0
3,1/25/20,0
4,1/26/20,0
...,...,...
688,12/10/21,6496142
689,12/11/21,6528894
690,12/12/21,6546010
691,12/13/21,6581433


In [35]:
def make_country_confirmed_df(country):
    url_time_confirmed = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
    df = pd.read_csv(url_time_confirmed)
    df = df.loc[df["Country/Region"] == country]
    df = df.drop(columns=["Province/State", "Country/Region",
                 "Lat", "Long"]).sum().reset_index(name="Confirmed")
    df = df.rename(columns={"index": "date"})
    return df

df = make_country_confirmed_df("Germany")

fig = px.line(
        df,
        x="date",
        y="Confirmed",
        template="plotly_dark",
        labels={"value": "Cases", "variable": "Confirmed", "date": "Date"},
        hover_data={"value": ":,", "variable": False, "date": False},
    )

fig.show()

ValueError: Value of 'hover_data_0' is not the name of a column in 'data_frame'. Expected one of ['date', 'Confirmed'] but received: value

In [25]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1)

fig.append_trace(go.Scatter(
    x=[3, 4, 5],
    y=[1000, 1100, 1200],
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=[2, 3, 4],
    y=[100, 110, 120],
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=[0, 1, 2],
    y=[10, 11, 12]
), row=3, col=1)


fig.update_layout(height=600, width=600, title_text="Stacked Subplots")
fig.show()

In [ ]:
url ='https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv?raw=true'
daily_df = pd.read_csv(url)
daily_df 
df = daily_df.drop(columns=["Province/State","Country/Region","Lat","Long"]).sum(skipna=True).reset_index(name="total")
df = df.rename(columns={'index':'date'})
df

In [97]:
iso_url='https://gist.githubusercontent.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv?raw=true'
df_iso_url = pd.read_csv(iso_url)
df_iso_url =df_iso_url[["Country","Alpha-3 code"]]
df_iso_url["Alpha-3 code"] = df_iso_url["Alpha-3 code"].str.replace('"', '')

df_iso_url = df_iso_url.rename(columns={'Country': 'Country_Region'})
df_iso_url = df_iso_url.rename(columns={'Alpha-3 code': 'code'})
countries_df_merged = pd.merge(countries_df, df_iso_url, on="Country_Region")
countries_df_merged
df_iso_url = pd.read_csv(iso_url)
df_iso_url

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,"""AF""","""AFG""","""4""","""33""","""65"""
1,Albania,"""AL""","""ALB""","""8""","""41""","""20"""
2,Algeria,"""DZ""","""DZA""","""12""","""28""","""3"""
3,American Samoa,"""AS""","""ASM""","""16""","""-14.3333""","""-170"""
4,Andorra,"""AD""","""AND""","""20""","""42.5""","""1.6"""
...,...,...,...,...,...,...
251,Wallis and Futuna,"""WF""","""WLF""","""876""","""-13.3""","""-176.2"""
252,Western Sahara,"""EH""","""ESH""","""732""","""24.5""","""-13"""
253,Yemen,"""YE""","""YEM""","""887""","""15""","""48"""
254,Zambia,"""ZM""","""ZMB""","""894""","""-15""","""30"""


In [ ]:
conditions = ["confirmed", "deaths", "recovered"]
def make_global_df():
    def make_global_df(condition):
        url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_{condition}_global.csv?raw=true'
        df = pd.read_csv(url)
        df = df.drop(["Province/State","Country/Region","Lat","Long"], axis =1).sum().reset_index(name=condition)
        df = df.rename(columns={'index':'date'})
        return df

    final_df = None 
    for condition in conditions: 
        condition_df = make_global_df(condition)
        if final_df is None:
            final_df = condition_df
        else:
            final_df = final_df.merge(condition_df)
    return final_df
make_global_df()

In [ ]:
df = pd.read_csv("data/time_confirmed.csv")
df = df.loc[df["Country/Region"] == "Vietnam"]
df = df.drop(columns = ["Province/State","Country/Region", "Lat", "Long"]).sum().reset_index(name="confirmed")
df = df.rename(columns = {'index':"date"})
df

In [ ]:
conditions = ["confirmed", "deaths", "recovered"]

def make_country_df(country):
    def make_df(condition):
        df = pd.read_csv(f"data/time_{condition}.csv")
        df = df.loc[df["Country/Region"] == country]
        df = df.drop(columns = ["Province/State","Country/Region", "Lat", "Long"]).sum().reset_index(name=condition)
        df = df.rename(columns = {'index':"date"})
        return df

    final_df = None 

    for condition in conditions: 
        condition_df = make_df(condition)
        if final_df is None:
            final_df = condition_df
        else:
            final_df = final_df.merge(condition_df)
    return final_df  

df = make_country_df("Germany")
df

In [ ]:

def make_global_df():
    def make_df(condition)
        df = pd.read_csv(f"data/time_{condition}.csv")
        df = daily_df.drop(["Province/State", "Country/Region",
                           "Lat", "Long"], axis=1).sum().reset_index(name=condition)
        df = df.rename(columns={'index': 'date'})
        return df
    final_df = None
    for condition in conditions:
        condition_df = make_global_df(condition)
        if final_df is None:
            final_df = condition_df
        else:
            final_df = final_df.merge(condition_df)
    return final_df   

##

The metrics method was developed in the mid-1980s to extract phenological features
from the AVHRR-based NDVI time-series [35,36]. At the same time, the idea of using vegetation
index time-series to extract spectral reflectance corresponding to a certain phenological stage was
proposed by Holben [37].Later, both approaches were merged by researchers from the Laboratory
for Global Remote Sensing Studies at the University of Maryland [38]. In their work, metrics were
calculated by extracting spectral information for specific phenological stages defined by the NDVI
annual dynamics. The multi-temporal metrics were widely used for forest monitoring at continental
and global scales using MODIS [39] and Landsat data [6,19,20,40].

In [29]:
daily_time_url ='https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
daily_time_df = pd.read_csv(daily_time_url)
daily_time_df = daily_time_df.drop(["Lat","Long"], axis =1)
daily_time_df_filled = daily_time_df.dropna(subset=['Province/State'])
daily_time_df_filled = daily_time_df_filled.groupby("Country/Region").sum()
daily_time_df_AuCaCh = daily_time_df_filled.loc[['Australia','Canada','China'], :]
daily_time_df_AuCaCh.reset_index()

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,12/4/21,12/5/21,12/6/21,12/7/21,12/8/21,12/9/21,12/10/21,12/11/21,12/12/21,12/13/21
0,Australia,0,0,0,0,4,5,5,6,9,...,217836,219121,220555,222260,223914,225625,227382,228925,230768,232738
1,Canada,0,0,0,0,1,1,2,2,2,...,1813304,1815686,1820693,1823674,1827339,1831341,1834569,1839770,1841672,1849762
2,China,548,643,920,1406,2075,2877,5509,6087,8141,...,111671,111741,111840,111919,112002,112072,112163,112239,112345,112423
